In [148]:
from SmartApi import SmartConnect
import os
from pyotp import TOTP
import urllib
import json
import pandas as pd
import datetime as dt

In [84]:
key_path = r"C:\Users\91931\Desktop\Keys"
os.chdir(key_path)
key_secret = open("angle_key.txt",'r').read().split()
api_key = key_secret[0]
username = key_secret[2]
pwd = key_secret[3]
totp = TOTP(key_secret[4]).now()

smartApi = SmartConnect(api_key)

data = smartApi.generateSession(username, pwd, totp)

[I 250104 15:41:20 smartConnect:121] in pool


In [85]:
instrument_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = urllib.request.urlopen(instrument_url)
instrument_list = json.loads(response.read())

In [50]:
def token_lookup(ticker, exchange="NSE"):
    for instrument in instrument_list:
        if instrument["name"] == ticker and instrument["exch_seg"] == exchange and instrument['symbol'].split('-')[-1] == "EQ" :
            return instrument["token"]

In [56]:
def symbol_lookup(token, exchange="NSE"):
    for instrument in instrument_list:
        if instrument["token"] == token and instrument["exch_seg"] == exchange and instrument['symbol'].split('-')[-1] == "EQ" :
            return instrument["name"]

In [154]:
def hist_data(ticker,duration,interval,exchange="NSE"):
    params = {
             "exchange": exchange,
             "symboltoken": token_lookup(ticker),
             "interval": interval,
             "fromdate": (dt.date.today() - dt.timedelta(duration)).strftime("%Y-%m-%d %H:%M"),
             "todate": dt.date.today().strftime("%Y-%m-%d %H:%M")
            }
    historical_data = smartApi.getCandleData(params)
    df_data = pd.DataFrame(historical_data["data"], 
                           columns = ["date","open","high","low","close","volume"])
    df_data.set_index("date",inplace=True)
    df_data.index = pd.to_datetime(df_data.index)
    df_data.index = df_data.index.tz_localize(None)
    return df_data

In [54]:
token_lookup("PPL")

'10297'

In [58]:
symbol_lookup("10285")

'AYMSYNTEX'

In [160]:
hist_data("HDFCBANK",30,"ONE_HOUR")

,open,high,low,close,volume
date,,,,,
2024-12-05 09:15:00,1859.90,1859.90,1845.0,1846.80,2018488
2024-12-05 10:15:00,1846.80,1850.00,1835.2,1841.35,1393010
2024-12-05 11:15:00,1841.35,1866.30,1840.0,1862.85,2463881
2024-12-05 12:15:00,1862.85,1869.50,1862.1,1868.65,2066436
2024-12-05 13:15:00,1868.25,1874.00,1862.2,1872.60,2410595
...,...,...,...,...,...
2025-01-03 11:15:00,1759.70,1768.00,1758.0,1760.10,1428052
2025-01-03 12:15:00,1760.55,1762.00,1750.5,1759.60,1716078
2025-01-03 13:15:00,1759.60,1760.85,1750.0,1752.90,1238251


In [162]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the website containing the data
url = "https://dhan.co/nse-fno-lot-size/"

# Send an HTTP request to the website
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, "html.parser")

# Find the table containing the data
table = soup.find("table", {"class": "table table-bod-stk-mrkt"})

# Initialize lists to store data
companies = []
symbols = []
lot_size_jan = []
lot_size_feb = []
lot_size_mar = []

# Extract table rows
rows = table.find("tbody").find_all("tr")
for row in rows:
    cols = row.find_all("td")
    companies.append(cols[0].text.strip())
    symbols.append(cols[1].text.strip())
    lot_size_jan.append(cols[2].text.strip())
    lot_size_feb.append(cols[3].text.strip())
    lot_size_mar.append(cols[4].text.strip())

# Create a DataFrame
data = {
    "Company Name": companies,
    "Symbol": symbols,
    "Lot Size (Jan 2025)": lot_size_jan,
    "Lot Size (Feb 2025)": lot_size_feb,
    "Lot Size (Mar 2025)": lot_size_mar,
}
df = pd.DataFrame(data)

# Save the data to an Excel file
file_name = "NSE_FO_Lot_Sizes.xlsx"
df.to_excel(file_name, index=False)

print(f"Data Successfully saved to {file_name}")


AttributeError: 'NoneType' object has no attribute 'find'

In [164]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Set up the Selenium WebDriver
driver = webdriver.Chrome()  # Ensure you have the ChromeDriver installed and in PATH

# Open the webpage
url = "https://dhan.co/nse-fno-lot-size/"
driver.get(url)

# Wait for the page to load
time.sleep(5)  # Adjust the delay as needed

# Locate the table
rows = driver.find_elements(By.CSS_SELECTOR, "table.table-bod-stk-mrkt tbody tr")

# Initialize lists to store data
companies = []
symbols = []
lot_size_jan = []
lot_size_feb = []
lot_size_mar = []

# Extract data from rows
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    companies.append(cols[0].text.strip())
    symbols.append(cols[1].text.strip())
    lot_size_jan.append(cols[2].text.strip())
    lot_size_feb.append(cols[3].text.strip())
    lot_size_mar.append(cols[4].text.strip())

# Close the WebDriver
driver.quit()

# Create a DataFrame
data = {
    "Company Name": companies,
    "Symbol": symbols,
    "Lot Size (Jan 2025)": lot_size_jan,
    "Lot Size (Feb 2025)": lot_size_feb,
    "Lot Size (Mar 2025)": lot_size_mar,
}
df = pd.DataFrame(data)

# Save the data to an Excel file
file_name = "NSE_FO_Lot_Sizes_Selenium.xlsx"
df.to_excel(file_name, index=False)

print(f"Data successfully saved to {file_name}")


Data successfully saved to NSE_FO_Lot_Sizes_Selenium.xlsx


In [ ]:
import pandas as pd
import yfinance as yf  # Install using `pip install yfinance`

# Step 1: Load the Excel file
file_path = "NSE_FO_Lot_Sizes_Selenium.xlsx"
df = pd.read_excel(file_path)

# Step 2: Fetch current prices using Yahoo Finance
def get_current_price(symbol):
    try:
        stock = yf.Ticker(symbol)
        return stock.history(period="1d")['Close'].iloc[-1]
  # Get the last closing price
    except:
        return None  # Return None if the price is unavailable

# Create a new column for current prices
df['Current Price'] = df['Symbol'].apply(get_current_price)

# Step 3: Save the updated Excel file
output_file = "NSE_FO_Lot_Sizes_with_Prices.xlsx"
df.to_excel(output_file, index=False)

print(f"Updated file saved as {output_file}")
